In [1]:
%pylab
import pandas as pd
import pickle
import os
print('complete')

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib
complete


In [2]:
# Task: find sequence of tokens where BSC-seq-IF gets it right but MV and HMMCrowd fail.
# Steps:
# 1. load gold
# 2. load text
# 3. load crowd annotations
# 3. load predictions from each method.
# 4. Scroll through gold, stop when we get a token where BSC is correct and others are wrong.
# 5. Map the token idxs to NER labels or IOB labels.
# 6. For the 10 previous and subsequent tokens, print:
# (a) the text, 
# (b) the crowd labels 
# (c) the gold and 
# (d) the predictions
# 8. ask user to save it. If no, continue scrolling to next candidate.
# 7. Use inkscape to turn this into a pretty diagram

In [7]:
golddir = '../../../data/bayesian_sequence_combination/data/ner/'

goldfile = os.path.join(golddir, 'task1_test_gt.csv')
gold = pd.read_csv(goldfile, header=None).values
print(gold.shape)

annofile = os.path.join(golddir, 'task1_test_annos.csv')
annos = pd.read_csv(annofile, header=None).values
print(annos.shape)

textfile = os.path.join(golddir, 'task1_test_text.csv')
text = pd.read_csv(textfile, header=None).values
print(text.shape)

(42975, 1)
(42975, 47)
(42975, 1)


In [21]:
resdir = '../../../data/bayesian_sequence_combination/output/good_results/ner-by-sentence/'
resfile = os.path.join(resdir, 'pred_started-2018-07-04-02-50-58-Nseen6045.csv') # 06-23-07-01-12-Nseen6045.csv')
res = pd.read_csv(resfile)
print(res.columns)
print(res.shape)
bronze = [] # list of runner up methods' predictions
bronze.append(res[" 'bac_seq_integrateIF_integrateLSTM'"].values.astype(int))

#bronze.append(res["# 'majority'"].values)
#bronze.append(res[" 'ds'"].values)

#resfile = os.path.join(resdir, 'pred_started-2018-06-24-07-48-06-Nseen6045.csv')
#res = pd.read_csv(resfile)
#print(res.columns)
#bronze.append(res["# 'HMM_crowd'"].values)
#bronze.append(res[" 'HMM_crowd_then_LSTM'"].values)

#resfile = os.path.join(resdir, 'pred_started-2018-06-23-05-20-51-Nseen6045.csv')
#res = pd.read_csv(resfile)
print(res.columns)
silver = res["# 'bac_seq_integrateIF'"].values.astype(int)

Index(['# 'bac_seq_integrateIF'', ' 'bac_seq_integrateIF_integrateLSTM'',
       ' 'bac_seq_integrateIF_integrateLSTM_atEnd'',
       ' 'bac_seq_integrateIF_then_LSTM''],
      dtype='object')
(82411, 4)
Index(['# 'bac_seq_integrateIF'', ' 'bac_seq_integrateIF_integrateLSTM'',
       ' 'bac_seq_integrateIF_integrateLSTM_atEnd'',
       ' 'bac_seq_integrateIF_then_LSTM''],
      dtype='object')


In [23]:
for i in range(len(gold)):
    
    matchfound = False
    
    if gold[i] == silver[i]:
        for b in bronze:
            if gold[i] != b[i]:
                print('Interesting data found. Type s to save, n to continue, x to exit: ')
                matchfound = True
                break

    if not matchfound:
        continue
    
    start = i - 5
    if start < 0:
        start = 0

    end = i + 5
    if end > len(gold):
        end = len(gold)

    live_workers = np.argwhere(np.any(annos[start:end] != -1, axis=0)).flatten()
    
    print('Index = %i to %i' % (start, end))
    
    for worker in live_workers:
        print(annos[start:end, worker].T)
    print(text[start:end].flatten())
    print(gold[start:end].flatten())
    print(silver[start:end].flatten())
    for b in bronze:
        print(b[start:end].flatten())

    input_data = input(">>>")
    print(input_data)
    
    if input_data == 'x':
        break
    if input_data == 's':
        
        with open('../error_analysis_method_comparison.txt', 'a') as fh:
            
            for worker in live_workers:
                fh.write(str(annos[start:end, worker].T))
                fh.write('\n')
            
            fh.write(str(text[start:end].flatten()))
            fh.write('\n')
            fh.write(str(gold[start:end].flatten()))
            fh.write('\n')
            fh.write(str(silver[start:end].flatten()))
            fh.write('\n')

            for b in bronze:
                fh.write(str(b[start:end].flatten()))
                fh.write('\n')
            fh.write('\n')
                    

Interesting data found. Type s to save, n to continue, x to exit: 
Index = 320 to 330
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 4 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
['among' 'those' 'called' '.' 'But' 'Currin' ',' 'who' 'is' 'advising']
[1 1 1 1 1 4 1 1 1 1]
[1 1 1 1 1 4 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
>>>

Interesting data found. Type s to save, n to continue, x to exit: 
Index = 433 to 443
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 4 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
['amnesty' 'application' 'before' 'that' '.' 'Currin' 'said' 'the' 'law'
 'had']
[1 1 1 1 1 4 1 1 1 1]
[1 1 1 1 1 4 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
>>>

Interesting data found. Type s to save, n to continue, x to exit: 
Index = 470 to 480
[1 1 1 1 1 1 1 1 1 1]
[2 1 1 1 1 2 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[1 1 2 0 0 0 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
['ANC' ',' 'de' 'Klerk' "'s" 'National' 'Party' 'and' 'the' 'right-wing']
[2 1 4 3 1 2 0 1 1 1]
[1 1 1 1 1 2 0 1 1 1]

KeyboardInterrupt: 

In [25]:
path = '../../../Desktop/good_with_better_comparison.csv'
cmp = pd.read_csv(path)

In [26]:
cmp.shape

(82411, 2)

In [27]:
np.argwhere(cmp[cmp.columns[0]] != cmp[cmp.columns[1]])

array([[  637],
       [  638],
       [  700],
       [  951],
       [ 1421],
       [ 1625],
       [ 1799],
       [ 1885],
       [ 2122],
       [ 2179],
       [ 2435],
       [ 2692],
       [ 2693],
       [ 3217],
       [ 3252],
       [ 3740],
       [ 3807],
       [ 4072],
       [ 4077],
       [ 4082],
       [ 4087],
       [ 4110],
       [ 4116],
       [ 4117],
       [ 4124],
       [ 4152],
       [ 4153],
       [ 4158],
       [ 4159],
       [ 4166],
       [ 4168],
       [ 4169],
       [ 4170],
       [ 4181],
       [ 4197],
       [ 4215],
       [ 4220],
       [ 4238],
       [ 4239],
       [ 4244],
       [ 4245],
       [ 4256],
       [ 4257],
       [ 4269],
       [ 4273],
       [ 4277],
       [ 4278],
       [ 4281],
       [ 4282],
       [ 4283],
       [ 4285],
       [ 4291],
       [ 4292],
       [ 4644],
       [ 4645],
       [ 4646],
       [ 4647],
       [ 5763],
       [ 5764],
       [ 5913],
       [ 6336],
       [ 6673],
       [

In [33]:
cmp[cmp.columns[0]][81610:81620]

81610    1
81611    8
81612    1
81613    1
81614    2
81615    0
81616    0
81617    1
81618    2
81619    1
Name: # 'bac_seq_integrateIF', dtype: int64

In [32]:
cmp[cmp.columns[1]][81610:81620]

81610    1
81611    8
81612    1
81613    1
81614    2
81615    1
81616    1
81617    1
81618    2
81619    1
Name: # 'bac_seq_integrateIF'.1, dtype: int64

In [35]:
path1 = '../../../data/bayesian_sequence_combination/data/ner/all_anno_gold_val.csv'
path2 = '../../../Downloads/all_anno_gold_val.csv'

In [37]:
import pandas as pd
annos1 = pd.read_csv(path1)
annos2 = pd.read_csv(path2)

In [38]:
annos1.shape

(40295, 52)

In [39]:
annos2.shape

(40295, 52)

In [40]:
import numpy as np
np.in1d(annos1.columns,annos2.columns)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [41]:
np.in1d(annos2.columns, annos1.columns)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [42]:
for col in annos1.columns:
    print col
    if col == 'text' or col == 'doc_id':
        continue
    print(np.any(np.invert((annos1[col] == annos2[col]) | (np.isnan(annos1[col]) & np.isnan(annos2[col]) ) )) )

Unnamed: 0
False
text
A1CZIM8L5T2KYP
False
doc_start
False
doc_id
tok_idx
False
AFM0E176VBXE2
False
AK0C1OVJLDGTZ
False
A2P7LS0DT6QM6X
False
A5H5MJI5GFN92
False
A3P7Y0914EBD6E
False
A7YBNPEFKN436
False
A2SB2BW2W4T4WW
False
A1K7JTHA3E7VHU
False
A6IVYRP8FAIS
False
A2FQSVTAOL5GUC
False
A3O6XMWN8W84L5
False
A2EPRPJA681I8X
False
A283KEXX5ACFWM
False
A2QX30PEG8W87U
False
A1W4TKCJ00KKM8
False
A3CD2802ITWXNH
False
A1GYFJ7J6KNOV7
False
A2O994ITMFDVB5
False
A31X3JCHS0BPFJ
False
A1C6BWZZF4IOR5
False
A254V0U3BGBPE5
False
A3FB1E807EN22S
False
A111JI6APXR6QV
False
A9DKHO6HZ966E
False
A2CQNGJAAPKQPI
False
A1XMLT8EP04IF1
False
A1W02OZN6J9J0H
False
AGWH82NB1FO3B
False
A2VJZZPAHHKAOW
False
A3OSHYHJX8BXHU
False
AKB1IT758U2LU
False
A1L09AVOUS47FK
False
A1HCL132B6I2U9
False
AC0BIEQG3Z6MD
False
ATIGCUKIPNWG0
False
A29X1A8AF3TUQP
False
A2ZUC3INRC7TP3
False
A2T19IADVP11HY
False
A1T9DFT0XOO3V5
False
A2TEWO3WJ5YKGG
False
A3IG0TV6T181YE
False
A9MYC5IGQ2DO4
False
A3M8JZIGN17WWT
False
A79CHO3NW2IPO
False
A3K6BSW10

In [29]:
col = 'AFM0E176VBXE2'
np.any(np.invert((annos1[col] == annos2[col]) | (np.isnan(annos1[col]) & np.isnan(annos2[col]) ) ))

False

In [44]:
annos1_sorted = annos1.reindex(sorted(annos1.columns), axis=1)

In [45]:
annos2_sorted = annos2.reindex(sorted(annos2.columns), axis=1)

In [52]:
np.any( (np.invert(annos1_sorted.isna()) | np.invert(annos2_sorted.isna()) ) & (annos1_sorted != annos2_sorted))

False